In [1]:
#importing the required libraries
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
def cars_search(search_term,location):
    
    loc = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    search_term = search_term.replace(' ','%20')
    url_frame = f'https://www.cars24.com/buy-used-car?sort=P&search={search_term}&storeCityId=8597&pinId={loc[location]}'
    
    return url_frame

In [3]:
def cars_scrapping():
      
    driver = webdriver.Chrome(r"F://chromedriver.exe")
    driver.maximize_window()
    
    loc = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    location = [keys for keys in loc.keys()]
    
    CARS= ['SUV', 'Sedan', 'Hatchback', 'Luxury Sedan', 'Luxury SUV']
    
    URL = [cars_search(j,i) for i in location for j in CARS]
    
    LOCATION = []
    YEAR = []
    BRAND = []
    MODEL = []
    VARIANT = []
    KM_DRIVEN = []
    NO_OF_OWNERS = []
    FUEL = []
    PRICE = []


    for i in URL:
        driver.get(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'class' : '_1l4fi'})

        description = []
        description1 = []
        description2 = []
        price = []
    
        for i in results:
            description.append(i.find('h2', {'class': "_3FpCg"}).text)
            description1.append(i.find('p', {'class': "cvakB"}).text)
            description2.append(i.find('ul', {'class': "bVR0c"},'li').text)
            price.append(i.find('div', {'class': "_7udZZ"}).text)
            LOCATION.append(driver.find_element_by_xpath("//p[@class='_1OR7d']/label").text)
    
    
        year = [i.split(' ',2)[0] for i in description]
        brand = [i.split(' ',2)[1] for i in description]
        D1= [i.split(' ',2)[-1] for i in description]

        for i in price:
            PRICE.append(i)

        for i in year:
            YEAR.append(i)
        for i in brand:
            BRAND.append(i)
    
        D2 = [i.rsplit(' ',1)[0] for i in description1]
        variant = [i.split(' ') [-1]for i in description1]
        for i in variant:
            VARIANT.append(i)
    
        D3 = [D1[i]+' '+D2[i] for i in range(0,len(D1))]
    
        for i in D3:
            MODEL.append(i)
    
        dummy1 = [i.split('km')[-1]for i in description2]
        kms = [i.split('km')[0] for i in description2]
        kms = [i+'km' for i in kms]

        owners = [i.split('Owner')[0] for i in dummy1]
        owners = [i+'Owner' for i in owners]
        fuel = [i.split('Owner')[-1] for i in dummy1]

        for i in kms:
            KM_DRIVEN.append(i)

        for i in owners:
            NO_OF_OWNERS.append(i)

        for i in fuel:
            FUEL.append(i)

    driver.close()
        
    used_cars = pd.DataFrame({"LOCATION" : LOCATION,
                              "YEAR" : YEAR,
                              "BRAND" : BRAND,
                              "MODEL" : MODEL,
                              "VARIANT" : VARIANT,
                              "KM_DRIVEN" : KM_DRIVEN,
                              "FUEL" : FUEL,
                              "NO_OF_OWNERS" : NO_OF_OWNERS,
                              "PRICE" : PRICE
                             })
    
    used_cars.to_csv("used_cars.csv",index=False) #Creating CSV
    print("SCRAPPED DETAILS ARE STORED AS 'used_cars.csv'")
    
    return used_cars

In [4]:
used_cars = cars_scrapping()

SCRAPPED DETAILS ARE STORED AS 'used_cars.csv'


In [5]:
used_cars

,LOCATION,YEAR,BRAND,MODEL,VARIANT,KM_DRIVEN,FUEL,NO_OF_OWNERS,PRICE
0,New Delhi,2019,Toyota,Innova Crysta 2.4 VX 7 STR,Manual,"31,235 km",Diesel,1st Owner,"₹18,00,000"
1,New Delhi,2015,Mahindra,TUV300 T8 AT,Automatic,"65,766 km",Diesel,1st Owner,"₹5,16,000"
2,New Delhi,2015,Maruti,Ertiga ZDI,Manual,"65,754 km",Diesel,1st Owner,"₹5,99,000"
3,New Delhi,2019,Renault,TRIBER 1.0 RXT,,"48,151 km",Petrol,1st Owner,"₹4,99,000"
4,New Delhi,2018,Maruti,Vitara Brezza VDI AMT,Manual,"28,944 km",Diesel,1st Owner,"₹8,15,000"
...,...,...,...,...,...,...,...,...,...
6446,Ahmedabad,2017,Honda,BR-V 1.5 i- DTEC S,Manual,"1,34,191 km",Diesel,1st Owner,"₹6,05,999"
6447,Ahmedabad,2017,Maruti,Eeco 5 STR,Manual,"67,514 km",Petrol,1st Owner,"₹2,98,299"
6448,Ahmedabad,2018,Honda,WR-V 1.2 i-VTEC VX MT,Manual,"24,484 km",Petrol,1st Owner,"₹7,44,399"
6449,Ahmedabad,2017,Maruti,Eeco 5 STR WITH AC PLUSHTR,Manual,"46,169 km",Petrol + CNG,2nd Owner,"₹3,93,499"
